In [9]:
import networkx as nx
import numpy as np

from qiskit import Aer
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.aqua.algorithms import QAOA, VQE, NumPyMinimumEigensolver
from qiskit.aqua.components.optimizers import SPSA, COBYLA
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.optimization.applications.ising import tsp
from qiskit.optimization.converters import IsingToQuadraticProgram
from qiskit.optimization.problems import QuadraticProgram
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer

from qiskit.circuit.library import TwoLocal

In [1]:

def get_graph(nodes):

    G = nx.Graph()
    G.add_nodes_from(np.arange(0, nodes, 1))

    # Create random positions in the graph. Distance will be calculated from positions
    # Note: Dwave and other solvers require a complete graph
    for i in range(nodes):
        G.nodes[i]['pos'] = (np.random.uniform(0, 10), np.random.uniform(0, 10))

    elist = set()
    for i in range(nodes):
        for t in range(i + 1,nodes):
            y1=G.nodes[i]['pos'][1]
            x1=G.nodes[i]['pos'][0]
            y2=G.nodes[t]['pos'][1]
            x2=G.nodes[t]['pos'][0]
            dist = np.sqrt(((x2-x1)**2)+((y2-y1)**2))
            _tuple = (i, t, dist)
            elist.add(_tuple)

    # tuple is (i,j,weight) where (i,j) is the edge
    G.add_weighted_edges_from(elist)

    return G

def get_cost_matrix(G, nodes):
    w = np.zeros([nodes,nodes])
    for i in range(nodes):
        for j in range(nodes):
            temp = G.get_edge_data(i,j,default=0)
            if temp != 0:
                w[i,j] = temp['weight']
    return w

def calculate_cost(cost_matrix, solution):
    cost = 0
    for i in range(len(solution)):
        a = i % len(solution)
        b = (i + 1) % len(solution)
        cost += cost_matrix[solution[a]][solution[b]]

    return cost



In [50]:
nodes = 4
starting_node = 0

####
G = get_graph(nodes)
cost_matrix = get_cost_matrix(G, nodes)

G.nodes[0]['pos']
coords = []
for node in G.nodes:
    coords.append(G.nodes[node]['pos'])
    print(G.nodes[node]['pos'])


tsp_instance = tsp.TspData(name = "TSP", dim = len(G.nodes), coord = coords, w = cost_matrix)
qubitOp, offset = tsp.get_operator(tsp_instance)
#print(qubitOp.print_details())
print("Qubits: ",qubitOp.num_qubits)

(6.816747921274889, 2.5615681229460208)
(7.1499054427197315, 1.1289044344800203)
(0.6120166099464819, 0.9809454946981899)
(7.246389298447326, 0.7693195764747729)
Qubits:  16


## First test with a common VQE and a TwoLocal ansatz
Use COBYLA to find right set of thetas for the circuit. Use VQE Qiskit function for simplicity.

We create the QUBO based on the Operator so we can compare different options. VQE directly using the Operator, or using the QUBO and any of the available Optimizers

In [51]:
qp = QuadraticProgram()
qp.from_ising(qubitOp, offset, linear=True)
qp.to_docplex().prettyprint()

// This file has been generated by DOcplex
// model name is: AnonymousModel
// single vars section
dvar bool x_0;
dvar bool x_1;
dvar bool x_2;
dvar bool x_3;
dvar bool x_4;
dvar bool x_5;
dvar bool x_6;
dvar bool x_7;
dvar bool x_8;
dvar bool x_9;
dvar bool x_10;
dvar bool x_11;
dvar bool x_12;
dvar bool x_13;
dvar bool x_14;
dvar bool x_15;

minimize
 - 200000 x_0 - 200000.000000 x_1 - 200000.000000 x_2 - 200000 x_3
 - 200000.000000 x_4 - 200000.000000 x_5 - 200000.000000 x_6
 - 200000.000000 x_7 - 200000.000000 x_8 - 200000.000000 x_9
 - 200000.000000 x_10 - 200000.000000 x_11 - 200000 x_12 - 200000.000000 x_13
 - 200000.000000 x_14 - 200000.000000 x_15 [ 200000 x_0*x_1 + 200000 x_0*x_2
 + 200000 x_0*x_3 + 200000 x_0*x_4 + 1.470891 x_0*x_5 + 1.470891 x_0*x_7
 + 200000 x_0*x_8 + 6.402895 x_0*x_9 + 6.402895 x_0*x_11 + 200000 x_0*x_12
 + 1.843026 x_0*x_13 + 1.843026 x_0*x_15 + 200000 x_1*x_2 + 200000 x_1*x_3
 + 1.470891 x_1*x_4 + 200000 x_1*x_5 + 1.470891 x_1*x_6 + 6.402895 x_1*x_8
 + 

## Try with VQE directly

Try using the VQE class from Qiskit directly and sending the operator created above. We can try creating our own variational form (TwoLocal or RealAmplitudes) and test possible results and hyperparameters on the optimizer. We can try COBYLA and SPSA as possible gradient algorithms. 


In [48]:
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend)

optimizer = SPSA(maxiter=400)
#optimizer = COBYLA(maxiter=100, rhobeg=2, tol=0.5, disp=True)
ry = TwoLocal(qubitOp.num_qubits, 'ry', 'cx', reps=3, entanglement='full')
vqe = VQE(operator=qubitOp, optimizer=optimizer, quantum_instance=quantum_instance)
result = vqe.run(quantum_instance)
print(result)

x = sample_most_likely(result.eigenstate)
print(x)
print(tsp.tsp_feasible(x))
if(tsp.tsp_feasible(x)):
    z = tsp.get_tsp_solution(x)
    print('solution with VQE:', z)
else:
    print('No solution found with VQE and such parameters')

{'optimal_parameters': {Parameter(θ[0]): -0.07495536394455037, Parameter(θ[10]): -2.1625540180918925, Parameter(θ[11]): 0.1741512713733136, Parameter(θ[12]): -0.16697426424280165, Parameter(θ[13]): 3.2091583492451288, Parameter(θ[14]): 1.2462116261506124, Parameter(θ[15]): -3.1110459266514687, Parameter(θ[16]): -1.609264286218274, Parameter(θ[17]): 0.31817658180472314, Parameter(θ[18]): 1.7221115235262388, Parameter(θ[19]): 1.145369129546944, Parameter(θ[1]): -3.8215647714396477, Parameter(θ[20]): -1.0551749459742692, Parameter(θ[21]): -2.387289018568104, Parameter(θ[22]): 1.6973042256691895, Parameter(θ[23]): 1.1913071069176124, Parameter(θ[24]): 1.9547930355259238, Parameter(θ[25]): -2.071401340658233, Parameter(θ[26]): -2.617484305769794, Parameter(θ[27]): -2.58840966344317, Parameter(θ[28]): 2.7439879897514765, Parameter(θ[29]): 1.8379698946032668, Parameter(θ[2]): -0.02510253425504253, Parameter(θ[30]): 1.0096909949130175, Parameter(θ[31]): 1.3361025604163022, Parameter(θ[32]): 1.

## Compare with the classical Eigensolver

In [30]:
npes = NumPyMinimumEigensolver(qubitOp)
result_classical = npes.run()
x = sample_most_likely(result_classical.eigenstate)
print(x)
if(tsp.tsp_feasible(x)):
    z = tsp.get_tsp_solution(x)
    print('solution Numpy classical solver:', z)
else:
    print('No solution found with Classical Eigensolver')

[1 0 0 0 1 0 0 0 1]
solution Numpy classical solver: [0, 1, 2]


In [40]:
## Try now with the optimizer and the Recursive Optimizer

backend = quantum_instance = Aer.get_backend('qasm_simulator')
optimizer = COBYLA(maxiter=200, rhobeg=3, tol=1.5, disp=True)

# MinimumEigenSolvers
qaoa_mes = QAOA(quantum_instance = backend, optimizer=optimizer)
vqe_mes = VQE(quantum_instance=backend, optimizer=optimizer, operator=qubitOp)
exact_mes = NumPyMinimumEigensolver()

# Optimizers
qaoa = MinimumEigenOptimizer(qaoa_mes)   # using QAOA
vqe = MinimumEigenOptimizer(vqe_mes)   # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

# We can try using VQE or QAOA for the minimum eigen optimizer and compare results
rqaoa = RecursiveMinimumEigenOptimizer(min_eigen_optimizer=vqe, min_num_vars=1, min_num_vars_optimizer=exact)
#rqaoa = RecursiveMinimumEigenOptimizer(min_eigen_optimizer=qaoa, min_num_vars=1, min_num_vars_optimizer=exact)


In [47]:
exact_result = exact.solve(qp)
print("Result Exact QP: ",exact_result.x)
if(tsp.tsp_feasible(exact_result.x)):
    print(tsp.get_tsp_solution(exact_result.x))
      
qaoa_result = qaoa.solve(qp)
print("Result QAOA with QP: ",qaoa_result.x)
if(tsp.tsp_feasible(qaoa_result.x)):
    print(tsp.get_tsp_solution(qaoa_result.x))

vqe_result = vqe.solve(qp)
print("Result VQE with QP: ",vqe_result.x)
if(tsp.tsp_feasible(vqe_result.x)):
    print(tsp.get_tsp_solution(vqe_result.x))
      
rqaoa_result = rqaoa.solve(qp)
print("Result RQAOA (withVQE) QP: ",rqaoa_result.x)
if(tsp.tsp_feasible(rqaoa_result.x)):
    print(tsp.get_tsp_solution(rqaoa_result.x))

# Compare with using VQE directly without optimizer. That is why we send qubit operator to the MES
result = vqe_mes.run(quantum_instance)
x = sample_most_likely(result_classical.eigenstate)
print("Result VQE with Operator: ",x)
if(tsp.tsp_feasible(x)):
    print(tsp.get_tsp_solution(x))


Result Exact QP:  [0. 0. 1. 0. 1. 0. 1. 0. 0.]
[2, 1, 0]
Result QAOA with QP:  [1. 0. 0. 0. 0. 1. 0. 1. 0.]
[0, 2, 1]
Result VQE with QP:  [1. 0. 0. 0. 1. 0. 0. 0. 1.]
[0, 1, 2]
Result RQAOA QP:  [1. 0. 0. 1. 0. 0. 0. 0. 1.]
Result VQE with Operator:  [1 0 0 0 1 0 0 0 1]
[0, 1, 2]


In [52]:
qubitOp.num_qubits


16